# 使用Subclass API构建动态模型

本教程介绍Keras的Model Subclassing API，这是最灵活但也最复杂的模型构建方式。

## 学习目标

1. 理解Model子类化的基本语法
2. 掌握自定义模型的构建方法
3. 学会在call方法中实现动态计算逻辑
4. 了解三种API的适用场景和权衡

## 三种API对比

| API类型 | 灵活性 | 调试难度 | 适用场景 |
|---------|--------|----------|----------|
| Sequential | 低 | 简单 | 线性堆叠模型 |
| Functional | 中 | 中等 | 静态多输入/输出 |
| Subclass | 高 | 复杂 | 动态计算图、研究 |

## 1. 环境配置与数据准备

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")

In [ ]:
# 加载并预处理数据
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, test_size=0.2, random_state=RANDOM_SEED
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=RANDOM_SEED
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# 准备多输入数据
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]

print(f"训练集: {X_train.shape}")
print(f"Wide输入: {X_train_A.shape}, Deep输入: {X_train_B.shape}")

## 2. Model Subclassing基础

### 核心概念

Model子类化需要：
1. 继承`keras.Model`类
2. 在`__init__`中定义层
3. 在`call`方法中实现前向传播逻辑

### 基本模板

```python
class MyModel(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # 在这里定义层
        self.dense1 = keras.layers.Dense(30, activation='relu')
        self.output_layer = keras.layers.Dense(1)
    
    def call(self, inputs, training=None):
        # 在这里定义前向传播逻辑
        x = self.dense1(inputs)
        return self.output_layer(x)
```

In [ ]:
# 简单的自定义模型示例
class SimpleRegressor(keras.Model):
    """
    简单的回归模型
    
    架构: Input -> Dense(30, ReLU) -> Dense(30, ReLU) -> Output
    """
    
    def __init__(self, units=30, activation='relu', **kwargs):
        """
        初始化模型层
        
        Parameters:
        -----------
        units : int
            隐藏层神经元数量
        activation : str
            激活函数
        """
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.output_layer = keras.layers.Dense(1)
    
    def call(self, inputs, training=None):
        """
        前向传播
        
        Parameters:
        -----------
        inputs : tensor
            输入张量
        training : bool, optional
            是否处于训练模式（用于Dropout等）
        
        Returns:
        --------
        tensor : 模型输出
        """
        x = self.hidden1(inputs)
        x = self.hidden2(x)
        return self.output_layer(x)

# 创建模型实例
simple_model = SimpleRegressor(units=30, name='simple_regressor')

# 编译模型
simple_model.compile(
    loss='mse',
    optimizer='sgd',
    metrics=['mae']
)

# 构建模型（指定输入形状）
simple_model.build(input_shape=(None, 8))
simple_model.summary()

In [ ]:
# 训练简单模型
history_simple = simple_model.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    verbose=1
)

# 评估
test_loss, test_mae = simple_model.evaluate(X_test, y_test, verbose=0)
print(f"\n测试集 MSE: {test_loss:.4f}, MAE: {test_mae:.4f}")

## 3. Wide & Deep模型（子类化版本）

使用子类化API实现多输入多输出的Wide & Deep模型。

In [ ]:
class WideAndDeepModel(keras.Model):
    """
    Wide & Deep模型的子类化实现
    
    架构:
    - Wide路径: 直接连接到输出
    - Deep路径: 两层隐藏层 + 辅助输出
    - 合并后产生主输出
    """
    
    def __init__(self, units=30, activation='relu', **kwargs):
        """
        初始化Wide & Deep模型
        
        Parameters:
        -----------
        units : int
            隐藏层神经元数量
        activation : str
            激活函数类型
        """
        super().__init__(**kwargs)
        
        # Deep路径的隐藏层
        self.hidden1 = keras.layers.Dense(units, activation=activation, name='deep_hidden1')
        self.hidden2 = keras.layers.Dense(units, activation=activation, name='deep_hidden2')
        
        # 输出层
        self.main_output = keras.layers.Dense(1, name='main_output')
        self.aux_output = keras.layers.Dense(1, name='aux_output')
        
        # 合并层
        self.concat = keras.layers.Concatenate(name='concat')
    
    def call(self, inputs, training=None):
        """
        前向传播
        
        Parameters:
        -----------
        inputs : tuple of tensors
            (wide_input, deep_input)
        training : bool, optional
            训练模式标志
        
        Returns:
        --------
        tuple : (main_output, aux_output)
        """
        # 解包输入
        input_wide, input_deep = inputs
        
        # Deep路径
        hidden1_out = self.hidden1(input_deep)
        hidden2_out = self.hidden2(hidden1_out)
        
        # 合并Wide和Deep路径
        concat_out = self.concat([input_wide, hidden2_out])
        
        # 计算输出
        main_out = self.main_output(concat_out)
        aux_out = self.aux_output(hidden2_out)
        
        return main_out, aux_out

# 创建模型
wide_deep_model = WideAndDeepModel(units=30, name='wide_and_deep')

# 编译模型
wide_deep_model.compile(
    loss=['mse', 'mse'],
    loss_weights=[0.9, 0.1],
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=['mae']
)

In [ ]:
# 训练Wide & Deep模型
history_wd = wide_deep_model.fit(
    (X_train_A, X_train_B), (y_train, y_train),
    epochs=30,
    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)),
    verbose=1
)

In [ ]:
# 评估模型
results = wide_deep_model.evaluate(
    (X_test_A, X_test_B), (y_test, y_test), verbose=0
)

print("评估结果:")
print(f"总损失: {results[0]:.4f}")
print(f"主输出损失: {results[1]:.4f}")
print(f"辅助输出损失: {results[2]:.4f}")

## 4. 动态行为示例

子类化API的最大优势是可以在`call`方法中实现动态计算逻辑，
例如条件分支、循环等Python控制流。

In [ ]:
class DynamicModel(keras.Model):
    """
    带有动态行为的模型示例
    
    特点:
    - 训练时使用Dropout
    - 可以根据输入动态选择计算路径
    """
    
    def __init__(self, units=30, dropout_rate=0.2, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation='relu')
        self.hidden2 = keras.layers.Dense(units, activation='relu')
        self.dropout = keras.layers.Dropout(dropout_rate)
        self.output_layer = keras.layers.Dense(1)
    
    def call(self, inputs, training=None):
        """
        前向传播，带有动态Dropout
        
        training参数在fit()时自动设为True，
        在evaluate()和predict()时自动设为False
        """
        x = self.hidden1(inputs)
        
        # Dropout只在训练时生效
        x = self.dropout(x, training=training)
        
        x = self.hidden2(x)
        x = self.dropout(x, training=training)
        
        return self.output_layer(x)

# 创建并训练动态模型
dynamic_model = DynamicModel(units=30, dropout_rate=0.2, name='dynamic_model')
dynamic_model.compile(loss='mse', optimizer='adam', metrics=['mae'])

history_dynamic = dynamic_model.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    verbose=1
)

# 评估
test_loss, test_mae = dynamic_model.evaluate(X_test, y_test, verbose=0)
print(f"\n测试集 MSE: {test_loss:.4f}, MAE: {test_mae:.4f}")

## 5. 自定义层

除了自定义模型，还可以创建自定义层，实现更细粒度的控制。

In [ ]:
class ResidualBlock(keras.layers.Layer):
    """
    残差块（Residual Block）自定义层
    
    实现: output = activation(input + Dense(Dense(input)))
    """
    
    def __init__(self, units, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
        
    def build(self, input_shape):
        """根据输入形状构建层的权重"""
        self.dense1 = keras.layers.Dense(self.units, activation='relu')
        self.dense2 = keras.layers.Dense(input_shape[-1])  # 输出维度与输入相同
        super().build(input_shape)
    
    def call(self, inputs):
        """前向传播：残差连接"""
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.activation(inputs + x)  # 残差连接
    
    def get_config(self):
        """返回层配置，用于模型序列化"""
        config = super().get_config()
        config.update({
            'units': self.units,
            'activation': keras.activations.serialize(self.activation)
        })
        return config

In [ ]:
# 使用自定义层构建模型
class ResNet(keras.Model):
    """使用残差块的模型"""
    
    def __init__(self, n_blocks=3, units=30, **kwargs):
        super().__init__(**kwargs)
        self.input_dense = keras.layers.Dense(units, activation='relu')
        self.res_blocks = [ResidualBlock(units) for _ in range(n_blocks)]
        self.output_layer = keras.layers.Dense(1)
    
    def call(self, inputs, training=None):
        x = self.input_dense(inputs)
        for block in self.res_blocks:
            x = block(x)
        return self.output_layer(x)

# 创建并训练ResNet
resnet = ResNet(n_blocks=3, units=30, name='resnet')
resnet.compile(loss='mse', optimizer='adam', metrics=['mae'])

history_resnet = resnet.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    verbose=1
)

test_loss, test_mae = resnet.evaluate(X_test, y_test, verbose=0)
print(f"\n测试集 MSE: {test_loss:.4f}, MAE: {test_mae:.4f}")

## 6. 可视化训练过程

In [ ]:
# 对比不同模型的训练曲线
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 损失曲线
axes[0].plot(history_simple.history['loss'], label='Simple')
axes[0].plot(history_dynamic.history['loss'], label='Dynamic (Dropout)')
axes[0].plot(history_resnet.history['loss'], label='ResNet')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Training Loss')
axes[0].set_title('Training Loss Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 验证损失
axes[1].plot(history_simple.history['val_loss'], label='Simple')
axes[1].plot(history_dynamic.history['val_loss'], label='Dynamic (Dropout)')
axes[1].plot(history_resnet.history['val_loss'], label='ResNet')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Validation Loss')
axes[1].set_title('Validation Loss Comparison')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. 注意事项与最佳实践

### 子类化API的优势

1. **完全的灵活性**: 可以实现任意复杂的前向传播逻辑
2. **动态计算图**: 支持条件分支、循环等控制流
3. **易于调试**: 可以在call方法中使用print或断点
4. **研究友好**: 适合实现新的模型架构

### 子类化API的限制

1. **无法自动推断形状**: 需要显式调用build()或传入数据
2. **序列化复杂**: 需要实现get_config()方法
3. **不支持某些Keras功能**: 如plot_model可能无法显示完整结构

### 选择建议

- **日常使用**: 优先使用Sequential或Functional API
- **复杂架构**: 静态图用Functional，动态图用Subclass
- **研究实验**: Subclass API最适合快速原型开发

## 小结

### 核心要点

1. **继承正确的基类**: `keras.Model`用于模型，`keras.layers.Layer`用于自定义层
2. **__init__定义组件**: 所有可训练层都应在初始化时创建
3. **call实现逻辑**: 前向传播的完整计算流程
4. **training参数**: 用于区分训练和推理模式

### 何时使用子类化

- 需要动态计算图（如循环神经网络的动态展开）
- 实现复杂的注意力机制
- 研究新的网络架构
- 需要在前向传播中使用Python控制流